In [11]:

import openai
import os
import json
import pandas as pd
pd.set_option('display.max_columns', 500)
from fuzzywuzzy import process
import numpy as np
import pulp
from typing import List
from pydantic import BaseModel, Field





# openai.api_key = secret



class Ingredient(BaseModel):
    name: str
    calories: int
    mass: str = Field(..., description="Ingredient mass in grams")
    food_group: str = Field(..., description="Type of food like meat, vegetable, cereals, dairy product")
class Recipe(BaseModel):
    meal_type: str = Field(..., description="Break fast, lunch, dinner or Snacks")
    title: str
    steps: List[str]
    meal_calories_sum: int = Field(..., description="sum of individual ingredient calories in meal")
    Ingrediets: List[Ingredient]
class daily_meal(BaseModel):
    Meal: List[Recipe]
class Ingredient_class:
    def __init__(self, name, protein, fat, carb):
        self.name = name
        self.protein = protein
        self.fat = fat
        self.carb = carb
class ingred_selection(BaseModel):
    ingreds_list: List[str] = Field(..., description="Ingredients that contribute significantly to protein, fat and carbohydrate macronutrients")

def cal_bmr(weight_kg, height_cm, age_years, gender, activity_factor):
    gender_fac = { 'Male':-5,
                  'Female':161
                 }
    moderate_fac = {
                    "Sedentary": 1.2,
                    "Lightly Active": 1.375,
                    "Moderately Active": 1.55,
                    "Very Active": 1.725,
                    "Extremely Active": 1.9
                    }
    return (            (10 * weight_kg) + (6.25 * height_cm) - (5 * age_years) - gender_fac.get(gender,-5)           )*moderate_fac.get(activity_factor,1.55)




def calculate_bmi(weight_kg, height_cm):
    # Convert height from centimeters to meters
    height_m = height_cm / 100

    # Calculate BMI
    bmi = weight_kg / (height_m ** 2)
    return bmi



def weight_from_bmi(bmi, height_cm):
    # Convert height from centimeters to meters
    height_m = height_cm / 100

    # Calculate BMI
    weight_kg = bmi * (height_m ** 2)
    return weight_kg


def goal_selection(weight_kg, height_cm):
    bmi = calculate_bmi(weight_kg, height_cm)
    if bmi>24.9:
        goal = weight_from_bmi(24.9, height_cm)
    elif bmi<18.5:
        goal = weight_from_bmi(18.5, height_cm)
    else:
        goal = weight_kg
    return goal-weight_kg


macros_ratio = {
  "Sedentary": {
    "Protein": 20,
    "Carbohydrates": 60,
    "Fat": 20
  },
  "Lightly Active": {
    "Protein": 25,
    "Carbohydrates": 50,
    "Fat": 25
  },
  "Moderately Active": {
    "Protein": 30,
    "Carbohydrates": 45,
    "Fat": 25
  },
  "Very Active": {
    "Protein": 35,
    "Carbohydrates": 40,
    "Fat": 25
  },
  "Extremely Active": {
    "Protein": 40,
    "Carbohydrates": 40,
    "Fat": 20
  }
}


def find_contributer_ingreds(ingreds):
    print('finding suitable ingreds')
    prompt_for_ingreds = f"I have ingredients {', '.join(set(ingreds))} please give the list of ingredients which are significant contributer of macronutrients protein, fat, carbohydrates in human diet. Exclude the ones like pepper, chilli etc are not significant contributer of macronutrients  or other ingredients whose change may effect meal quality"
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
       {"role": "user", "content": prompt_for_ingreds}
            ],
        functions=[
            {
              "name": "get_significant_macros_ingredients",
              "description": "significant contributer of macronutrients protein, fat, carbohydrates in human diet.",
              "parameters": ingred_selection.schema()
            }
        ],
        function_call={"name": "get_significant_macros_ingredients"}
        )
    ingreds_adjustable = json.loads(response.choices[0]["message"]["function_call"]["arguments"]).get('ingreds_list')
    return ingreds_adjustable




def adjust_ingredients(ingredients, required_protein, required_fat, required_carb, min_max_bounds):
    prob = pulp.LpProblem("MacronutrientAdjustment", pulp.LpMinimize)



    ingredient_vars = {ingredient: pulp.LpVariable(ingredient.name,
    min_max_bounds[ingredient.name])
    for ingredient in ingredients}



    protein_diff = pulp.LpVariable("protein_diff", 0)
    fat_diff = pulp.LpVariable("fat_diff", 0)
    carb_diff = pulp.LpVariable("carb_diff", 0)



    # Objective: Minimize the total difference from the required macros
    prob += protein_diff + fat_diff + carb_diff, "TotalDifference"



    # Constraints to compute the difference
    prob += sum(ingredient.protein * ingredient_vars[ingredient] for ingredient in ingredients) - required_protein <= protein_diff, "ProteinDifference1"
    prob += required_protein - sum(ingredient.protein * ingredient_vars[ingredient] for ingredient in ingredients) <= protein_diff, "ProteinDifference2"



    prob += sum(ingredient.fat * ingredient_vars[ingredient] for ingredient in ingredients) - required_fat <= fat_diff, "FatDifference1"
    prob += required_fat - sum(ingredient.fat * ingredient_vars[ingredient] for ingredient in ingredients) <= fat_diff, "FatDifference2"



    prob += sum(ingredient.carb * ingredient_vars[ingredient] for ingredient in ingredients) - required_carb <= carb_diff, "CarbDifference1"
    prob += required_carb - sum(ingredient.carb * ingredient_vars[ingredient] for ingredient in ingredients) <= carb_diff, "CarbDifference2"



    prob.solve()



    results = {}
    for ingredient in ingredients:
        results[ingredient.name] = pulp.value(ingredient_vars[ingredient])
    return results


def verify_results(adjusted_ingredients, ingredients_data, required_protein, required_fat, required_carb, tolerance_percent=5):
    total_protein = 0
    total_fat = 0
    total_carb = 0
    for ingredient in ingredients_data:
        amount = adjusted_ingredients.get(ingredient.name, 0)
        total_protein += ingredient.protein * amount
        total_fat += ingredient.fat * amount
        total_carb += ingredient.carb * amount

    protein_range = (required_protein * (1 - tolerance_percent/100), required_protein * (1 + tolerance_percent/100))
    fat_range = (required_fat * (1 - tolerance_percent/100), required_fat * (1 + tolerance_percent/100))
    carb_range = (required_carb * (1 - tolerance_percent/100), required_carb * (1 + tolerance_percent/100))
    # print('verification')
    # print(protein_range[0] <= total_protein <= protein_range[1])
    # print(total_protein)
    # print(fat_range[0] <= total_fat <= fat_range[1])
    # print(total_fat)
    # print(carb_range[0] <= total_carb <= carb_range[1])
    # print(total_carb)
    if protein_range[0] <= total_protein <= protein_range[1] and         fat_range[0] <= total_fat <= fat_range[1] and         carb_range[0] <= total_carb <= carb_range[1]:
        return True
    return False



def ratios_adjustment_data_frame(diet_frame_gpt, calories_gpt):
    diet_frame_gpt.loc[:, 'pr_cl'] = diet_frame_gpt['Protein (g)'] * 4
    diet_frame_gpt.loc[:, 'ft_cl'] = diet_frame_gpt['Fat (g)'] * 9
    diet_frame_gpt.loc[:, 'cb_cl'] = diet_frame_gpt['Carbohydrate (g)'] * 4
    diet_frame_gpt['prot_rt'] = diet_frame_gpt['pr_cl']/(diet_frame_gpt['ft_cl']+diet_frame_gpt['pr_cl']+diet_frame_gpt['cb_cl'])
    diet_frame_gpt['fat_rt'] = diet_frame_gpt['ft_cl']/(diet_frame_gpt['ft_cl']+diet_frame_gpt['pr_cl']+diet_frame_gpt['cb_cl'])
    diet_frame_gpt['carb_rt'] = diet_frame_gpt['cb_cl']/(diet_frame_gpt['ft_cl']+diet_frame_gpt['pr_cl']+diet_frame_gpt['cb_cl'])
    ingreds = list(diet_frame_gpt['name'])
    ingreds_adjustable = find_contributer_ingreds(ingreds)
    diet_frame = pd.DataFrame(diet_frame_gpt.loc[diet_frame_gpt['name'].isin(ingreds_adjustable)].reset_index(drop=True))
    ingredients_data = [Ingredient_class(diet_frame['name'][i],
                                   diet_frame['prot_rt'][i],
                                   diet_frame['fat_rt'][i],
                                   diet_frame['carb_rt'][i])
                        for i in range(len(diet_frame))]
    calories = sum(diet_frame['calories'])
    required_protein = (macros_ratio[activity_factor]['Protein']/100)*(calories)
    required_fat = (macros_ratio[activity_factor]['Fat']/100)*(calories)
    required_carb = (macros_ratio[activity_factor]['Carbohydrates']/100)*(calories)
    min_max_bounds = {diet_frame['name'][i]: diet_frame['calories'][i]*0.5
                        for i in range(len(diet_frame))}
    tolerance = 5
    adjusted_ingredients = adjust_ingredients(ingredients_data,
                                              required_protein, required_fat, required_carb,
                                              min_max_bounds)
    for name, amount in adjusted_ingredients.items():
        diet_frame_gpt.loc[diet_frame_gpt['name']==name, 'calories'] = amount
        print(f"{name}: {amount:.2f} calories")
    # Verify results
    if verify_results(adjusted_ingredients, ingredients_data, required_protein, required_fat, required_carb):
        print("\nAchieved macros are within the desired range!")
    else:
        print("\nAchieved macros are not within the desired range.")

    diet_frame_gpt['1cal_gm'] = 100/diet_frame_gpt['Calories']
    temp_cal = sum(diet_frame_gpt['calories'])
    diet_frame_gpt['cal_ratio'] = diet_frame_gpt['calories']/temp_cal
    diet_frame_gpt['required_calories'] = ((calories)*diet_frame_gpt['cal_ratio'])
    diet_frame_gpt['per_meal_gms'] = diet_frame_gpt['required_calories'] * diet_frame_gpt['1cal_gm']
    diet_frame_gpt['taste'] = ''
    diet_frame_gpt.loc[diet_frame_gpt['required_calories']==0.0, 'taste'] = 'As per taste'
    df = pd.DataFrame(diet_frame_gpt[['Name', 'per_meal_gms']])
    result = []
    for index, row in df.iterrows():
        result.append({k: v for k, v in row.items()})
    return result


# Function to get user input with default values
def get_user_input(prompt, default_value):
    try:
        user_input = input(prompt)
        if user_input:
            return user_input
        else:
            return default_value
    except Exception:
        return default_value




def main(gender, height_cm, weight_kg, age_years, \
    diet_type, activity_factor, avoid, include, \
    cuisine_type, max_number_of_ingredients, \
    max_cooking_time, country_available, request, \
    number_of_meals, time_of_goal):
    how_much_weight_kg_diff_goal = goal_selection(weight_kg, height_cm)
    # BMR Calculation (Mifflin-St. Jeor equation)
    d_bmr = cal_bmr(weight_kg, height_cm, age_years,  gender, activity_factor)


    cal_deficit_total = how_much_weight_kg_diff_goal*7700 #1kg for 7700 calories and 1 pound for 3500

    if time_of_goal == 0:
        time_of_goal = int(abs((cal_deficit_total/3500)*7))


    cal_deficit_daily = cal_deficit_total/time_of_goal

    required_calories_daily =  int(d_bmr+cal_deficit_daily)

    # Display the results
    print(f"User needs to take total of {required_calories_daily} in a day to achieve goal within {time_of_goal} days.")


    if diet_type == 'paleo diet':
        prompt = f"Generate a recipe containing {required_calories_daily} calories according to paleo diet."
    elif diet_type == 'keto diet':
        prompt = f"Generate a recipe containing {required_calories_daily} calories according to keto diet."
    else:
        prompt = f"Generate a recipe containing {required_calories_daily} calories with ratio of protein={macros_ratio[activity_factor]['Protein']}, carbohydrates={macros_ratio[activity_factor]['Carbohydrates']} and fats={macros_ratio[activity_factor]['Fat']}."

    prompt+=f"Daily meal plan has to be generated including {number_of_meals} meals in a day. "
    prompt+=f"Ingredients should be available in {country_available} and suitable for cuisine type {cuisine_type}. Ingredients included in recipe should not exceed count of {max_number_of_ingredients} with maximum cooking time {max_cooking_time} minutes. And diet should follow special request of client ({request})."
    if len(avoid)>0:
        prompt+=f" Recipe should avoid {', '.join(avoid)}."
    if len(include)>0:
        prompt+=f" Recipe should include {', '.join(include)}."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
        {"role": "user", "content": prompt}
        ],
        functions=[
            {
            "name": "get_steps_and_ingredients_query",
            "description": "Get user answer in series of steps",
            "parameters": daily_meal.schema()
            }
        ],
        function_call={"name": "get_steps_and_ingredients_query"}
    )

    output = json.loads(response.choices[0]["message"]["function_call"]["arguments"])
    # output = [{'meal_type': 'breakfast', 'title': 'Greek Yogurt Bowl', 'steps': ['In a bowl, mix together Greek yogurt, honey, and vanilla extract.', 'Top the yogurt mixture with granola, berries, and sliced almonds.', 'Serve immediately.'], 'meal_calories_sum': 600, 'Ingrediets': [{'name': 'Greek yogurt', 'calories': 150, 'mass': '100g', 'food_group': 'Dairy'}, {'name': 'Honey', 'calories': 60, 'mass': '2 tbsp', 'food_group': 'Condiments'}, {'name': 'Vanilla extract', 'calories': 10, 'mass': '1 tsp', 'food_group': 'Condiments'}, {'name': 'Granola', 'calories': 150, 'mass': '1/4 cup', 'food_group': 'Grains'}, {'name': 'Berries', 'calories': 60, 'mass': '1/4 cup', 'food_group': 'Fruits'}, {'name': 'Sliced almonds', 'calories': 170, 'mass': '2 tbsp', 'food_group': 'Nuts and Seeds'}]}, {'meal_type': 'lunch', 'title': 'Quinoa Salad', 'steps': ['Cook quinoa according to package instructions and let it cool.', 'In a large bowl, combine quinoa, diced cucumbers, diced bell peppers, chopped parsley, and crumbled feta cheese.', 'In a small bowl, whisk together olive oil, lemon juice, garlic powder, salt, and pepper to make the dressing.', 'Pour the dressing over the quinoa mixture and toss to combine.', 'Serve chilled.'], 'meal_calories_sum': 800, 'Ingrediets': [{'name': 'Quinoa', 'calories': 120, 'mass': '1/2 cup', 'food_group': 'Grains'}, {'name': 'Cucumbers', 'calories': 20, 'mass': '1/2 cup', 'food_group': 'Vegetables'}, {'name': 'Bell peppers', 'calories': 30, 'mass': '1/2 cup', 'food_group': 'Vegetables'}, {'name': 'Parsley', 'calories': 10, 'mass': '2 tbsp', 'food_group': 'Herbs'}, {'name': 'Feta cheese', 'calories': 120, 'mass': '1/4 cup', 'food_group': 'Dairy'}, {'name': 'Olive oil', 'calories': 120, 'mass': '2 tbsp', 'food_group': 'Oils'}, {'name': 'Lemon juice', 'calories': 10, 'mass': '1 tbsp', 'food_group': 'Condiments'}, {'name': 'Garlic powder', 'calories': 10, 'mass': '1 tsp', 'food_group': 'Condiments'}, {'name': 'Salt', 'calories': 0, 'mass': 'to taste', 'food_group': 'Condiments'}, {'name': 'Pepper', 'calories': 0, 'mass': 'to taste', 'food_group': 'Condiments'}]}, {'meal_type': 'dinner', 'title': 'Grilled Chicken with Vegetables', 'steps': ['Preheat grill to medium heat.', 'Season chicken breasts with salt, pepper, and your choice of seasonings.', 'Grill chicken for 6-8 minutes on each side or until cooked through.', 'In a separate grill pan, cook sliced zucchini, bell peppers, and onions with olive oil, salt, and pepper until tender.', 'Serve grilled chicken with grilled vegetables.'], 'meal_calories_sum': 1000, 'Ingrediets': [{'name': 'Chicken breasts', 'calories': 200, 'mass': '1 piece', 'food_group': 'Meat'}, {'name': 'Salt', 'calories': 0, 'mass': 'to taste', 'food_group': 'Condiments'}, {'name': 'Pepper', 'calories': 0, 'mass': 'to taste', 'food_group': 'Condiments'}, {'name': 'Seasonings', 'calories': 0, 'mass': 'to taste', 'food_group': 'Condiments'}, {'name': 'Zucchini', 'calories': 20, 'mass': '1/2 cup', 'food_group': 'Vegetables'}, {'name': 'Bell peppers', 'calories': 30, 'mass': '1/2 cup', 'food_group': 'Vegetables'}, {'name': 'Onions', 'calories': 10, 'mass': '1/4 cup', 'food_group': 'Vegetables'}, {'name': 'Olive oil', 'calories': 120, 'mass': '2 tbsp', 'food_group': 'Oils'}]}]
    print('gpt out put:\n', output)


    food_db_org = pd.read_csv("/content/drive/MyDrive/diet-project/food_db.csv")
    ignored = list(food_db_org.loc[(food_db_org['200 Calorie Weight (g)'].isnull()) | (food_db_org['200 Calorie Weight (g)'] == 0.0) | (food_db_org['Calories'].isnull()) | (food_db_org['Calories'] == 0.0) ]['Name'])
    food_db = food_db_org.loc[~((food_db_org['200 Calorie Weight (g)'].isnull()) | (food_db_org['200 Calorie Weight (g)'] == 0.0) | (food_db_org['Calories'].isnull()) | (food_db_org['Calories'] == 0.0)) ]



    meals_list = []
    for meal in output:
        diet_frame = pd.DataFrame()
        for ingreient in meal['Ingrediets']:
            query = ingreient['name']

            best_match, score, index = process.extractOne(query, food_db['Name'])
            ingreient['similarity_score'] = score
            if (score < 50) or (best_match not in ignored):
                best_match, score, index = process.extractOne(ingreient['food_group'], food_db['Food Group'])
            result = pd.concat([pd.DataFrame(ingreient, index=[0]),
                                food_db.iloc[[index]][['Name','Calories', 'Food Group', 'Fat (g)', 'Protein (g)', 'Carbohydrate (g)', '200 Calorie Weight (g)']].reset_index(drop=True)
                            ], axis=1)
            diet_frame = pd.concat([diet_frame, result], axis=0)
        diet_frame.reset_index(drop=True, inplace=True)
        meals_list.append({'calories': meal['meal_calories_sum'],
                        'meal_type': meal['meal_type'],
                        'title': meal['title'],
                        'steps': meal['steps'],
                        'data_frame': diet_frame})

    updated_meal_data_frames = {}
    for diet_frame in meals_list:
        updated_meal_data_frames[diet_frame['meal_type']] = \
        {'title': diet_frame['title'],
        'steps': diet_frame['steps'],
        'Ingrediets': ratios_adjustment_data_frame(diet_frame_gpt = diet_frame['data_frame'], calories_gpt = diet_frame['calories'])}
    return updated_meal_data_frames




if __name__ == '__main__':
    # Default values
    gender = "Male"
    height_cm = 170
    weight_kg = 81
    age_years = 26
    diet_type = 'ratio based diet'
    activity_factor = 'Moderately Active'
    avoid = ['Tomatoes']
    include = ['Greek Yogurt']
    cuisine_type = 'any'
    max_number_of_ingredients = 5
    max_cooking_time = 30
    country_available = 'globally'
    request = 'diet should almost be balanced'
    number_of_meals = 3
    time_of_goal = 90
    use_recommended_time = 1

    # Assign values based on user input or default values
    gender = get_user_input("Enter your gender (Male/Female): ", gender)
    height_cm = int(get_user_input("Enter your height in cm: ", height_cm))
    weight_kg = int(get_user_input("Enter your weight in kg: ", weight_kg))
    age_years = int(get_user_input("Enter your age in years: ", age_years))
    diet_type = get_user_input("Enter your diet type: ", diet_type)
    activity_factor = get_user_input("Enter your activity factor: ", activity_factor)
    avoid = get_user_input("Enter foods to avoid (comma-separated): ", ','.join(avoid)).split(',')
    include = get_user_input("Enter foods to include (comma-separated): ", ','.join(include)).split(',')
    cuisine_type = get_user_input("Enter your preferred cuisine type: ", cuisine_type)
    max_number_of_ingredients = int(get_user_input("Enter max number of ingredients: ", max_number_of_ingredients))
    max_cooking_time = int(get_user_input("Enter max cooking time in minutes: ", max_cooking_time))
    country_available = get_user_input("Enter available countries: ", country_available)
    request = get_user_input("Enter your dietary request: ", request)
    number_of_meals = int(get_user_input("Enter number of meals between 2 to 4 in a day: ", number_of_meals))
    use_recommended_time = int(get_user_input("Enter 1 if want to use recommended time or 2 if want to use self defined time period", use_recommended_time))
    if use_recommended_time and use_recommended_time == 1:
        time_of_goal = 0
    else:
        time_of_goal = int(get_user_input("Enter your goal time in days: ", time_of_goal))
    meal_plan_suggested = main(gender, height_cm, weight_kg, age_years, \
                               diet_type, activity_factor, avoid, include, \
                               cuisine_type, max_number_of_ingredients, \
                               max_cooking_time, country_available, request, \
                               number_of_meals, time_of_goal)
    print("meal_plan_suggested:\n", meal_plan_suggested)


Enter your gender (Male/Female): 
Enter your height in cm: 
Enter your weight in kg: 
Enter your age in years: 
Enter your diet type: 
Enter your activity factor: 
Enter foods to avoid (comma-separated): 
Enter foods to include (comma-separated): 
Enter your preferred cuisine type: 
Enter max number of ingredients: 
Enter max cooking time in minutes: 
Enter available countries: 
Enter your dietary request: 
Enter number of meals between 2 to 4 in a day: 
Enter 1 if want to use recommended time or 2 if want to use self defined time period
User needs to take total of 2207 in a day to achieve goal within 139 days.
gpt out put:
 [{'meal_type': 'Breakfast', 'title': 'Greek Yogurt Parfait', 'steps': ['In a glass or bowl, layer Greek yogurt, granola, and mixed berries.', 'Repeat the layers until all ingredients are used.', 'Serve immediately or refrigerate for later.'], 'meal_calories_sum': 700, 'Ingrediets': [{'name': 'Greek yogurt', 'calories': 150, 'mass': '1 cup', 'food_group': 'Dairy'}, 

In [6]:


!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 23.9 MB/s eta 0:00:00


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fuzzywuzzy

In [ ]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 75.4 MB/s eta 0:00:00


In [ ]:

import openai
import os
import json
import pandas as pd
pd.set_option('display.max_columns', 500)
from fuzzywuzzy import process
import numpy as np
import pulp
from typing import List
from pydantic import BaseModel, Field





# secret = secret
openai.api_key = secret



class Ingredient(BaseModel):
    name: str
    calories: int
    mass: str = Field(..., description="Ingredient mass in grams")
    food_group: str = Field(..., description="Type of food like meat, vegetable, cereals, dairy product")
class Recipe(BaseModel):
    meal_type: str = Field(..., description="Break fast, lunch, dinner or Snacks")
    title: str
    steps: List[str]
    meal_calories_sum: int = Field(..., description="sum of individual ingredient calories in meal")
    Ingrediets: List[Ingredient]
class daily_meal(BaseModel):
    Meal: List[Recipe]
class Ingredient_class:
    def __init__(self, name, protein, fat, carb):
        self.name = name
        self.protein = protein
        self.fat = fat
        self.carb = carb
class ingred_selection(BaseModel):
    ingreds_list: List[str] = Field(..., description="Ingredients that contribute significantly to protein, fat and carbohydrate macronutrients")

def cal_bmr(weight_kg, height_cm, age_years, gender, activity_factor):
    gender_fac = { 'Male':-5,
                  'Female':161
                 }
    moderate_fac = {
                    "Sedentary": 1.2,
                    "Lightly Active": 1.375,
                    "Moderately Active": 1.55,
                    "Very Active": 1.725,
                    "Extremely Active": 1.9
                    }
    return (            (10 * weight_kg) + (6.25 * height_cm) - (5 * age_years) - gender_fac.get(gender,-5)           )*moderate_fac.get(activity_factor,1.55)




def calculate_bmi(weight_kg, height_cm):
    # Convert height from centimeters to meters
    height_m = height_cm / 100

    # Calculate BMI
    bmi = weight_kg / (height_m ** 2)
    return bmi



def weight_from_bmi(bmi, height_cm):
    # Convert height from centimeters to meters
    height_m = height_cm / 100

    # Calculate BMI
    weight_kg = bmi * (height_m ** 2)
    return weight_kg


def goal_selection(weight_kg, height_cm):
    bmi = calculate_bmi(weight_kg, height_cm)
    if bmi>24.9:
        goal = weight_from_bmi(24.9, height_cm)
    elif bmi<18.5:
        goal = weight_from_bmi(18.5, height_cm)
    else:
        goal = weight_kg
    return goal-weight_kg


macros_ratio = {
  "Sedentary": {
    "Protein": 20,
    "Carbohydrates": 60,
    "Fat": 20
  },
  "Lightly Active": {
    "Protein": 25,
    "Carbohydrates": 50,
    "Fat": 25
  },
  "Moderately Active": {
    "Protein": 30,
    "Carbohydrates": 45,
    "Fat": 25
  },
  "Very Active": {
    "Protein": 35,
    "Carbohydrates": 40,
    "Fat": 25
  },
  "Extremely Active": {
    "Protein": 40,
    "Carbohydrates": 40,
    "Fat": 20
  }
}


def find_contributer_ingreds(ingreds):
    print('finding suitable ingreds')
    prompt_for_ingreds = f"I have ingredients {', '.join(set(ingreds))} please give the list of ingredients which are significant contributer of macronutrients protein, fat, carbohydrates in human diet. Exclude the ones like pepper, chilli etc are not significant contributer of macronutrients  or other ingredients whose change may effect meal quality"
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
       {"role": "user", "content": prompt_for_ingreds}
            ],
        functions=[
            {
              "name": "get_significant_macros_ingredients",
              "description": "significant contributer of macronutrients protein, fat, carbohydrates in human diet.",
              "parameters": ingred_selection.schema()
            }
        ],
        function_call={"name": "get_significant_macros_ingredients"}
        )
    ingreds_adjustable = json.loads(response.choices[0]["message"]["function_call"]["arguments"]).get('ingreds_list')
    return ingreds_adjustable




def adjust_ingredients(ingredients, required_protein, required_fat, required_carb, min_max_bounds):
    prob = pulp.LpProblem("MacronutrientAdjustment", pulp.LpMinimize)



    ingredient_vars = {ingredient: pulp.LpVariable(ingredient.name,
    min_max_bounds[ingredient.name])
    for ingredient in ingredients}



    protein_diff = pulp.LpVariable("protein_diff", 0)
    fat_diff = pulp.LpVariable("fat_diff", 0)
    carb_diff = pulp.LpVariable("carb_diff", 0)



    # Objective: Minimize the total difference from the required macros
    prob += protein_diff + fat_diff + carb_diff, "TotalDifference"



    # Constraints to compute the difference
    prob += sum(ingredient.protein * ingredient_vars[ingredient] for ingredient in ingredients) - required_protein <= protein_diff, "ProteinDifference1"
    prob += required_protein - sum(ingredient.protein * ingredient_vars[ingredient] for ingredient in ingredients) <= protein_diff, "ProteinDifference2"



    prob += sum(ingredient.fat * ingredient_vars[ingredient] for ingredient in ingredients) - required_fat <= fat_diff, "FatDifference1"
    prob += required_fat - sum(ingredient.fat * ingredient_vars[ingredient] for ingredient in ingredients) <= fat_diff, "FatDifference2"



    prob += sum(ingredient.carb * ingredient_vars[ingredient] for ingredient in ingredients) - required_carb <= carb_diff, "CarbDifference1"
    prob += required_carb - sum(ingredient.carb * ingredient_vars[ingredient] for ingredient in ingredients) <= carb_diff, "CarbDifference2"



    prob.solve()



    results = {}
    for ingredient in ingredients:
        results[ingredient.name] = pulp.value(ingredient_vars[ingredient])
    return results


def verify_results(adjusted_ingredients, ingredients_data, required_protein, required_fat, required_carb, tolerance_percent=5):
    total_protein = 0
    total_fat = 0
    total_carb = 0
    for ingredient in ingredients_data:
        amount = adjusted_ingredients.get(ingredient.name, 0)
        total_protein += ingredient.protein * amount
        total_fat += ingredient.fat * amount
        total_carb += ingredient.carb * amount

    protein_range = (required_protein * (1 - tolerance_percent/100), required_protein * (1 + tolerance_percent/100))
    fat_range = (required_fat * (1 - tolerance_percent/100), required_fat * (1 + tolerance_percent/100))
    carb_range = (required_carb * (1 - tolerance_percent/100), required_carb * (1 + tolerance_percent/100))
    # print('verification')
    # print(protein_range[0] <= total_protein <= protein_range[1])
    # print(total_protein)
    # print(fat_range[0] <= total_fat <= fat_range[1])
    # print(total_fat)
    # print(carb_range[0] <= total_carb <= carb_range[1])
    # print(total_carb)
    if protein_range[0] <= total_protein <= protein_range[1] and         fat_range[0] <= total_fat <= fat_range[1] and         carb_range[0] <= total_carb <= carb_range[1]:
        return True
    return False



def ratios_adjustment_data_frame(diet_frame_gpt, calories_gpt):
    diet_frame_gpt.loc[:, 'pr_cl'] = diet_frame_gpt['Protein (g)'] * 4
    diet_frame_gpt.loc[:, 'ft_cl'] = diet_frame_gpt['Fat (g)'] * 9
    diet_frame_gpt.loc[:, 'cb_cl'] = diet_frame_gpt['Carbohydrate (g)'] * 4
    diet_frame_gpt['prot_rt'] = diet_frame_gpt['pr_cl']/(diet_frame_gpt['ft_cl']+diet_frame_gpt['pr_cl']+diet_frame_gpt['cb_cl'])
    diet_frame_gpt['fat_rt'] = diet_frame_gpt['ft_cl']/(diet_frame_gpt['ft_cl']+diet_frame_gpt['pr_cl']+diet_frame_gpt['cb_cl'])
    diet_frame_gpt['carb_rt'] = diet_frame_gpt['cb_cl']/(diet_frame_gpt['ft_cl']+diet_frame_gpt['pr_cl']+diet_frame_gpt['cb_cl'])
    ingreds = list(diet_frame_gpt['name'])
    ingreds_adjustable = find_contributer_ingreds(ingreds)
    diet_frame = pd.DataFrame(diet_frame_gpt.loc[diet_frame_gpt['name'].isin(ingreds_adjustable)].reset_index(drop=True))
    ingredients_data = [Ingredient_class(diet_frame['name'][i],
                                   diet_frame['prot_rt'][i],
                                   diet_frame['fat_rt'][i],
                                   diet_frame['carb_rt'][i])
                        for i in range(len(diet_frame))]
    calories = sum(diet_frame['calories'])
    required_protein = (macros_ratio[activity_factor]['Protein']/100)*(calories)
    required_fat = (macros_ratio[activity_factor]['Fat']/100)*(calories)
    required_carb = (macros_ratio[activity_factor]['Carbohydrates']/100)*(calories)
    min_max_bounds = {diet_frame['name'][i]: diet_frame['calories'][i]*0.5
                        for i in range(len(diet_frame))}
    tolerance = 5
    adjusted_ingredients = adjust_ingredients(ingredients_data,
                                              required_protein, required_fat, required_carb,
                                              min_max_bounds)
    for name, amount in adjusted_ingredients.items():
        diet_frame_gpt.loc[diet_frame_gpt['name']==name, 'calories'] = amount
        print(f"{name}: {amount:.2f} calories")
    # Verify results
    if verify_results(adjusted_ingredients, ingredients_data, required_protein, required_fat, required_carb):
        print("\nAchieved macros are within the desired range!")
    else:
        print("\nAchieved macros are not within the desired range.")

    diet_frame_gpt['1cal_gm'] = 100/diet_frame_gpt['Calories']
    temp_cal = sum(diet_frame_gpt['calories'])
    diet_frame_gpt['cal_ratio'] = diet_frame_gpt['calories']/temp_cal
    diet_frame_gpt['required_calories'] = ((calories)*diet_frame_gpt['cal_ratio'])
    diet_frame_gpt['per_meal_gms'] = diet_frame_gpt['required_calories'] * diet_frame_gpt['1cal_gm']
    diet_frame_gpt['taste'] = ''
    diet_frame_gpt.loc[diet_frame_gpt['required_calories']==0.0, 'taste'] = 'As per taste'
    df = pd.DataFrame(diet_frame_gpt[['Name', 'per_meal_gms']])
    result = []
    for index, row in df.iterrows():
        result.append({k: v for k, v in row.items()})
    return result


# Function to get user input with default values
def get_user_input(prompt, default_value):
    try:
        user_input = input(prompt)
        if user_input:
            return user_input
        else:
            return default_value
    except Exception:
        return default_value




def main(gender, height_cm, weight_kg, age_years, \
    diet_type, activity_factor, avoid, include, \
    cuisine_type, max_number_of_ingredients, \
    max_cooking_time, country_available, request, \
    number_of_meals, time_of_goal):
    how_much_weight_kg_diff_goal = goal_selection(weight_kg, height_cm)
    # BMR Calculation (Mifflin-St. Jeor equation)
    d_bmr = cal_bmr(weight_kg, height_cm, age_years,  gender, activity_factor)


    cal_deficit_total = how_much_weight_kg_diff_goal*7700 #1kg for 7700 calories and 1 pound for 3500

    if time_of_goal == 0:
        time_of_goal = int(abs((cal_deficit_total/3500)*7))


    cal_deficit_daily = cal_deficit_total/time_of_goal

    required_calories_daily =  int(d_bmr+cal_deficit_daily)

    # Display the results
    print(f"User needs to take total of {required_calories_daily} in a day to achieve goal within {time_of_goal} days.")


    if diet_type == 'paleo diet':
        prompt = f"Generate a recipe containing {required_calories_daily} calories according to paleo diet."
    elif diet_type == 'keto diet':
        prompt = f"Generate a recipe containing {required_calories_daily} calories according to keto diet."
    else:
        prompt = f"Generate a recipe containing {required_calories_daily} calories with ratio of protein={macros_ratio[activity_factor]['Protein']}, carbohydrates={macros_ratio[activity_factor]['Carbohydrates']} and fats={macros_ratio[activity_factor]['Fat']}."

    prompt+=f"Daily meal plan has to be generated including {number_of_meals} meals in a day. "
    prompt+=f"Ingredients should be available in {country_available} and suitable for cuisine type {cuisine_type}. Ingredients included in recipe should not exceed count of {max_number_of_ingredients} with maximum cooking time {max_cooking_time} minutes. And diet should follow special request of client ({request})."
    if len(avoid)>0:
        prompt+=f" Recipe should avoid {', '.join(avoid)}."
    if len(include)>0:
        prompt+=f" Recipe should include {', '.join(include)}."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
        {"role": "user", "content": prompt}
        ],
        functions=[
            {
            "name": "get_steps_and_ingredients_query",
            "description": "Get user answer in series of steps",
            "parameters": daily_meal.schema()
            }
        ],
        function_call={"name": "get_steps_and_ingredients_query"}
    )

    output = json.loads(response.choices[0]["message"]["function_call"]["arguments"])
    # output = [{'meal_type': 'breakfast', 'title': 'Greek Yogurt Bowl', 'steps': ['In a bowl, mix together Greek yogurt, honey, and vanilla extract.', 'Top the yogurt mixture with granola, berries, and sliced almonds.', 'Serve immediately.'], 'meal_calories_sum': 600, 'Ingrediets': [{'name': 'Greek yogurt', 'calories': 150, 'mass': '100g', 'food_group': 'Dairy'}, {'name': 'Honey', 'calories': 60, 'mass': '2 tbsp', 'food_group': 'Condiments'}, {'name': 'Vanilla extract', 'calories': 10, 'mass': '1 tsp', 'food_group': 'Condiments'}, {'name': 'Granola', 'calories': 150, 'mass': '1/4 cup', 'food_group': 'Grains'}, {'name': 'Berries', 'calories': 60, 'mass': '1/4 cup', 'food_group': 'Fruits'}, {'name': 'Sliced almonds', 'calories': 170, 'mass': '2 tbsp', 'food_group': 'Nuts and Seeds'}]}, {'meal_type': 'lunch', 'title': 'Quinoa Salad', 'steps': ['Cook quinoa according to package instructions and let it cool.', 'In a large bowl, combine quinoa, diced cucumbers, diced bell peppers, chopped parsley, and crumbled feta cheese.', 'In a small bowl, whisk together olive oil, lemon juice, garlic powder, salt, and pepper to make the dressing.', 'Pour the dressing over the quinoa mixture and toss to combine.', 'Serve chilled.'], 'meal_calories_sum': 800, 'Ingrediets': [{'name': 'Quinoa', 'calories': 120, 'mass': '1/2 cup', 'food_group': 'Grains'}, {'name': 'Cucumbers', 'calories': 20, 'mass': '1/2 cup', 'food_group': 'Vegetables'}, {'name': 'Bell peppers', 'calories': 30, 'mass': '1/2 cup', 'food_group': 'Vegetables'}, {'name': 'Parsley', 'calories': 10, 'mass': '2 tbsp', 'food_group': 'Herbs'}, {'name': 'Feta cheese', 'calories': 120, 'mass': '1/4 cup', 'food_group': 'Dairy'}, {'name': 'Olive oil', 'calories': 120, 'mass': '2 tbsp', 'food_group': 'Oils'}, {'name': 'Lemon juice', 'calories': 10, 'mass': '1 tbsp', 'food_group': 'Condiments'}, {'name': 'Garlic powder', 'calories': 10, 'mass': '1 tsp', 'food_group': 'Condiments'}, {'name': 'Salt', 'calories': 0, 'mass': 'to taste', 'food_group': 'Condiments'}, {'name': 'Pepper', 'calories': 0, 'mass': 'to taste', 'food_group': 'Condiments'}]}, {'meal_type': 'dinner', 'title': 'Grilled Chicken with Vegetables', 'steps': ['Preheat grill to medium heat.', 'Season chicken breasts with salt, pepper, and your choice of seasonings.', 'Grill chicken for 6-8 minutes on each side or until cooked through.', 'In a separate grill pan, cook sliced zucchini, bell peppers, and onions with olive oil, salt, and pepper until tender.', 'Serve grilled chicken with grilled vegetables.'], 'meal_calories_sum': 1000, 'Ingrediets': [{'name': 'Chicken breasts', 'calories': 200, 'mass': '1 piece', 'food_group': 'Meat'}, {'name': 'Salt', 'calories': 0, 'mass': 'to taste', 'food_group': 'Condiments'}, {'name': 'Pepper', 'calories': 0, 'mass': 'to taste', 'food_group': 'Condiments'}, {'name': 'Seasonings', 'calories': 0, 'mass': 'to taste', 'food_group': 'Condiments'}, {'name': 'Zucchini', 'calories': 20, 'mass': '1/2 cup', 'food_group': 'Vegetables'}, {'name': 'Bell peppers', 'calories': 30, 'mass': '1/2 cup', 'food_group': 'Vegetables'}, {'name': 'Onions', 'calories': 10, 'mass': '1/4 cup', 'food_group': 'Vegetables'}, {'name': 'Olive oil', 'calories': 120, 'mass': '2 tbsp', 'food_group': 'Oils'}]}]
    print('gpt out put:\n', output)


    food_db_org = pd.read_csv("/content/drive/MyDrive/diet-project/food_db.csv")
    ignored = list(food_db_org.loc[(food_db_org['200 Calorie Weight (g)'].isnull()) | (food_db_org['200 Calorie Weight (g)'] == 0.0) | (food_db_org['Calories'].isnull()) | (food_db_org['Calories'] == 0.0) ]['Name'])
    food_db = food_db_org.loc[~((food_db_org['200 Calorie Weight (g)'].isnull()) | (food_db_org['200 Calorie Weight (g)'] == 0.0) | (food_db_org['Calories'].isnull()) | (food_db_org['Calories'] == 0.0)) ]



    meals_list = []
    for meal in output:
        diet_frame = pd.DataFrame()
        for ingreient in meal['Ingrediets']:
            query = ingreient['name']

            best_match, score, index = process.extractOne(query, food_db['Name'])
            ingreient['similarity_score'] = score
            if (score < 50) or (best_match not in ignored):
                best_match, score, index = process.extractOne(ingreient['food_group'], food_db['Food Group'])
            result = pd.concat([pd.DataFrame(ingreient, index=[0]),
                                food_db.iloc[[index]][['Name','Calories', 'Food Group', 'Fat (g)', 'Protein (g)', 'Carbohydrate (g)', '200 Calorie Weight (g)']].reset_index(drop=True)
                            ], axis=1)
            diet_frame = pd.concat([diet_frame, result], axis=0)
        diet_frame.reset_index(drop=True, inplace=True)
        meals_list.append({'calories': meal['meal_calories_sum'],
                        'meal_type': meal['meal_type'],
                        'title': meal['title'],
                        'steps': meal['steps'],
                        'data_frame': diet_frame})

    updated_meal_data_frames = {}
    for diet_frame in meals_list:
        updated_meal_data_frames[diet_frame['meal_type']] = \
        {'title': diet_frame['title'],
        'steps': diet_frame['steps'],
        'Ingrediets': ratios_adjustment_data_frame(diet_frame_gpt = diet_frame['data_frame'], calories_gpt = diet_frame['calories'])}
    return updated_meal_data_frames




# Default values
gender = "Male"
height_cm = 170
weight_kg = 81
age_years = 26
diet_type = 'ratio based diet'
activity_factor = 'Moderately Active'
avoid = ['Tomatoes']
include = ['Greek Yogurt']
cuisine_type = 'any'
max_number_of_ingredients = 5
max_cooking_time = 30
country_available = 'globally'
request = 'diet should almost be balanced'
number_of_meals = 3
time_of_goal = 90
use_recommended_time = 1

In [ ]:
how_much_weight_kg_diff_goal = goal_selection(weight_kg, height_cm)

In [ ]:
how_much_weight_kg_diff_goal

-9.039000000000016

In [ ]:
d_bmr = cal_bmr(weight_kg, height_cm, age_years,  gender, activity_factor)

In [ ]:
d_bmr

2708.625

In [ ]:
cal_deficit_total = how_much_weight_kg_diff_goal*7700

In [ ]:
cal_deficit_total

-69600.30000000012

In [ ]:
if time_of_goal == 0:
    time_of_goal = int(abs((cal_deficit_total/3500)*7))

In [ ]:
time_of_goal

90

In [ ]:
required_calories_daily =  int(d_bmr+cal_deficit_daily)
required_calories_daily


1935

In [ ]:
print(f"User needs to take total of {required_calories_daily} in a day to achieve goal within {time_of_goal} days.")


User needs to take total of 1935 in a day to achieve goal within 90 days.


In [ ]:
prompt = f"Generate a recipe containing {required_calories_daily} calories with ratio of protein={macros_ratio[activity_factor]['Protein']}, carbohydrates={macros_ratio[activity_factor]['Carbohydrates']} and fats={macros_ratio[activity_factor]['Fat']}."
prompt+=f"Daily meal plan has to be generated including {number_of_meals} meals in a day. "
prompt+=f"Ingredients should be available in {country_available} and suitable for cuisine type {cuisine_type}. Ingredients included in recipe should not exceed count of {max_number_of_ingredients} with maximum cooking time {max_cooking_time} minutes. And diet should follow special request of client ({request})."


In [ ]:
prompt

'Generate a recipe containing 1935 calories with ratio of protein=30, carbohydrates=45 and fats=25.Daily meal plan has to be generated including 3 meals in a day. Ingredients should be available in globally and suitable for cuisine type any. Ingredients included in recipe should not exceed count of 5 with maximum cooking time 30 minutes. And diet should follow special request of client (diet should almost be balanced).'

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
    {"role": "user", "content": prompt}
    ],
    functions=[
        {
        "name": "get_steps_and_ingredients_query",
        "description": "Get user answer in series of steps",
        "parameters": daily_meal.schema()
        }
    ],
    function_call={"name": "get_steps_and_ingredients_query"}
)

In [ ]:
output = json.loads(response.choices[0]["message"]["function_call"]["arguments"])

In [ ]:
from pprint import pprint
pprint(output)

[{'Ingrediets': [{'calories': 70,
                  'food_group': 'Protein',
                  'mass': '2 large',
                  'name': 'Eggs'},
                 {'calories': 110,
                  'food_group': 'Fat',
                  'mass': '1/4 cup',
                  'name': 'Cheese'},
                 {'calories': 10,
                  'food_group': 'Vegetables',
                  'mass': '1 cup',
                  'name': 'Spinach'},
                 {'calories': 25,
                  'food_group': 'Fruit',
                  'mass': '1 medium',
                  'name': 'Tomatoes'}],
  'meal_calories_sum': 500,
  'meal_type': 'breakfast',
  'steps': ['Heat a non-stick skillet over medium heat.',
            'In a bowl, whisk together the eggs, salt, and pepper.',
            'Pour the egg mixture into the skillet and cook until set, about '
            '3-4 minutes.',
            'Fold the omelette in half and serve hot.'],
  'title': 'Omelette'},
 {'Ingrediets': [{'calorie

In [ ]:
food_db_org = pd.read_csv("/content/drive/MyDrive/diet-project/food_db.csv")

In [ ]:
food_db_org

,Unnamed: 0,ID,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score (),Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)",4:0 (mg),6:0 (mg),8:0 (mg),10:0 (mg),12:0 (mg),14:0 (mg),16:0 (mg),18:0 (mg),20:0 (mg),18:1 (mg),18:2 (mg),18:3 (mg),20:4 (mg),22:6 n-3 (DHA) (mg),22:0 (mg),14:1 (mg),16:1 (mg),18:4 (mg),20:1 (mg),20:5 n-3 (EPA) (mg),22:1 (mg),22:5 n-3 (DPA) (mg),Phytosterols (mg),Stigmasterol (mg),Campesterol (mg),Beta-sitosterol (mg),15:0 (mg),17:0 (mg),24:0 (mg),16:1 t (mg),18:1 t (mg),22:1 t (mg),18:2 t not further defined (mg),18:2 i (mg),"18:2 t,t (mg)",18:2 CLAs (mg),24:1 c (mg),"20:2 n-6 c,c (mg)",16:1 c (mg),18:1 c (mg),"18:2 n-6 c,c (mg)",22:1 c (mg),"18:3 n-6 c,c,c (mg)",17:1 (mg),20:3 (mg),"Fatty acids, total trans-monoenoic (mg)","Fatty acids, total trans-polyenoic (mg)",13:0 (mg),15:1 (mg),"18:3 n-3 c,c,c (ALA) (mg)",20:3 n-3 (mg),20:3 n-6 (mg),20:4 n-6 (mg),18:3i (mg),21:5 (mg),22:4 (mg),18:1-11 t (18:1t n-7) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg),Theobromine (mg),Serving Weight 1 (g),Serving Description 1 (g),Serving Weight 2 (g),Serving Description 2 (g),Serving Weight 3 (g),Serving Description 3 (g),Serving Weight 4 (g),Serving Description 4 (g),Serving Weight 5 (g),Serving Description 5 (g),Serving Weight 6 (g),Serving Description 6 (g),Serving Weight 7 (g),Serving Description 7 (g),Serving Weight 8 (g),Serving Description 8 (g),Serving Weight 9 (g),Serving Description 9 (g),200 Calorie Weight (g)
0,0,167512,Pillsbury Golden Layer Buttermilk Biscuits Art...,Baked Foods,307.0,13.24,5.88,41.18,5.88,1.2,0.0,2.941,NaN,2.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.98,35.50,NaN,NaN,NaN,4.412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1059.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0,1 serving,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.147
1,1,167513,Pillsbury Cinnamon Rolls With Icing Refrigerat...,Baked Foods,330.0,11.27,4.34,53.42,21.34,1.4,0.0,3.250,28.0,1.93,NaN,NaN,1.0,0.0,0.1,NaN,NaN,NaN,NaN,52.02,27.86,NaN,NaN,NaN,4.290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
ignored = list(food_db_org.loc[(food_db_org['200 Calorie Weight (g)'].isnull()) | (food_db_org['200 Calorie Weight (g)'] == 0.0) | (food_db_org['Calories'].isnull()) | (food_db_org['Calories'] == 0.0) ]['Name'])
ignored

['Sweetener Herbal Extract Powder From Stevia Leaf', 'Low Calorie Cola', 'Decaf Coffee', 'Tea Green Brewed Decaffeinated', 'Green Tea', 'Water Bottled Non-Carbonated Evian', 'Powerade Zero Ion4 Calorie-Free Assorted Flavors', 'Tea Hibiscus Brewed', 'Seasoning Mix Dry Sazon Coriander & Annatto', 'Babyfood Water Bottled Gerber Without Added Fluoride.', 'Gerolsteiner Brunnen Gmbh & Co. Kg (Gerolsteiner) Naturally Sparkling Mineral Bottled Water', 'The Coca-Cola Company Glaceau Vitamin Water Revive Fruit Punch Fortified', 'Carbonated Low Calorie Other Than Cola Or Pepper  Without Caffeine', 'Carbonated Low Calorie Other Than Cola Or Pepper With Aspartame Contains Caffeine', 'Tea Black Ready To Drink Decaffeinated Diet', 'Water Bottled Perrier', 'Water Bottled Poland Spring', 'Table Salt', 'Tap Water', 'Water Bottled Non-Carbonated Dannon', 'Water Bottled Non-Carbonated Pepsi Aquafina', 'Water Bottled Non-Carbonated Naya', 'Water Bottled Non-Carbonated Dannon Fluoride To Go', 'Mixed Berry P

In [ ]:
food_db = food_db_org.loc[~((food_db_org['200 Calorie Weight (g)'].isnull()) | (food_db_org['200 Calorie Weight (g)'] == 0.0) | (food_db_org['Calories'].isnull()) | (food_db_org['Calories'] == 0.0)) ]
food_db

,Unnamed: 0,ID,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score (),Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)",4:0 (mg),6:0 (mg),8:0 (mg),10:0 (mg),12:0 (mg),14:0 (mg),16:0 (mg),18:0 (mg),20:0 (mg),18:1 (mg),18:2 (mg),18:3 (mg),20:4 (mg),22:6 n-3 (DHA) (mg),22:0 (mg),14:1 (mg),16:1 (mg),18:4 (mg),20:1 (mg),20:5 n-3 (EPA) (mg),22:1 (mg),22:5 n-3 (DPA) (mg),Phytosterols (mg),Stigmasterol (mg),Campesterol (mg),Beta-sitosterol (mg),15:0 (mg),17:0 (mg),24:0 (mg),16:1 t (mg),18:1 t (mg),22:1 t (mg),18:2 t not further defined (mg),18:2 i (mg),"18:2 t,t (mg)",18:2 CLAs (mg),24:1 c (mg),"20:2 n-6 c,c (mg)",16:1 c (mg),18:1 c (mg),"18:2 n-6 c,c (mg)",22:1 c (mg),"18:3 n-6 c,c,c (mg)",17:1 (mg),20:3 (mg),"Fatty acids, total trans-monoenoic (mg)","Fatty acids, total trans-polyenoic (mg)",13:0 (mg),15:1 (mg),"18:3 n-3 c,c,c (ALA) (mg)",20:3 n-3 (mg),20:3 n-6 (mg),20:4 n-6 (mg),18:3i (mg),21:5 (mg),22:4 (mg),18:1-11 t (18:1t n-7) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg),Theobromine (mg),Serving Weight 1 (g),Serving Description 1 (g),Serving Weight 2 (g),Serving Description 2 (g),Serving Weight 3 (g),Serving Description 3 (g),Serving Weight 4 (g),Serving Description 4 (g),Serving Weight 5 (g),Serving Description 5 (g),Serving Weight 6 (g),Serving Description 6 (g),Serving Weight 7 (g),Serving Description 7 (g),Serving Weight 8 (g),Serving Description 8 (g),Serving Weight 9 (g),Serving Description 9 (g),200 Calorie Weight (g)
0,0,167512,Pillsbury Golden Layer Buttermilk Biscuits Art...,Baked Foods,307.0,13.24,5.88,41.18,5.88,1.2,0.0,2.941,NaN,2.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.98,35.50,NaN,NaN,NaN,4.412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1059.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0,1 serving,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.147
1,1,167513,Pillsbury Cinnamon Rolls With Icing Refrigerat...,Baked Foods,330.0,11.27,4.34,53.42,21.34,1.4,0.0,3.250,28.0,1.93,NaN,NaN,1.0,0.0,0.1,NaN,NaN,NaN,NaN,52.02,27.86,NaN,NaN,NaN,4.290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:

# meals_list = []
# for meal in output:
#     diet_frame = pd.DataFrame()
#     for ingreient in meal['Ingrediets']:
#         query = ingreient['name']

#         best_match, score, index = process.extractOne(query, food_db['Name'])
#         ingreient['similarity_score'] = score
#         if (score < 50) or (best_match not in ignored):
#             best_match, score, index = process.extractOne(ingreient['food_group'], food_db['Food Group'])
#         result = pd.concat([pd.DataFrame(ingreient, index=[0]),
#                             food_db.iloc[[index]][['Name','Calories', 'Food Group', 'Fat (g)', 'Protein (g)', 'Carbohydrate (g)', '200 Calorie Weight (g)']].reset_index(drop=True)
#                         ], axis=1)
#         diet_frame = pd.concat([diet_frame, result], axis=0)
#     diet_frame.reset_index(drop=True, inplace=True)
#     meals_list.append({'calories': meal['meal_calories_sum'],
#                     'meal_type': meal['meal_type'],
#                     'title': meal['title'],
#                     'steps': meal['steps'],
#                     'data_frame': diet_frame})


import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cdist
import re

# Preprocess food database

# Create TF-IDF vectorizer for character embeddings
char_vectorizer = TfidfVectorizer(analyzer='word', lowercase=True)
food_names_cleaned = [re.sub(r'[^a-zA-Z0-9\s]', '', name.lower()) for name in food_db['Name']]
char_vectorizer.fit(food_names_cleaned)
food_name_embeddings_array = char_vectorizer.transform(food_names_cleaned).toarray()

# Define a function to find the best match for an ingredient
def find_best_match(ingredients, ignored):
    batch_size = len(ingredients)
    query_cleaned = [re.sub(r'[^a-zA-Z0-9\s]', '', ingredient['name'].lower()) for ingredient in ingredients]
    query_embeddings = char_vectorizer.transform(query_cleaned).toarray()
    similarity_matrix = 1-cdist(query_embeddings, food_name_embeddings_array, metric='cosine')
    best_match_indices = np.argmax(similarity_matrix, axis=1)

    diet_frame = pd.DataFrame()

    for i in range(batch_size):
        best_match_index = best_match_indices[i]
        similarity_score = similarity_matrix[i][best_match_index]
        if similarity_score > 0.0:
            # food_name = food_db.iloc[best_match_index]['Name']
            result = pd.concat([pd.DataFrame(ingredients[i], index=[0]),
                                food_db.iloc[[best_match_index]][['Name','Calories', 'Food Group', 'Fat (g)', 'Protein (g)', 'Carbohydrate (g)', '200 Calorie Weight (g)']].reset_index(drop=True)
                            ], axis=1)
            diet_frame = pd.concat([diet_frame, result], axis=0)
            # best_match_info = food_db.iloc[best_match_index][['Name', 'Calories', 'Food Group', 'Fat (g)', 'Protein (g)', 'Carbohydrate (g)', '200 Calorie Weight (g)']]
            # diet_frame.append(diet_frame)

    return diet_frame.reset_index(drop=True)

# Process meals and ingredients efficiently
meals_list = []
diet_frame = pd.DataFrame()
for meal in output:
    ingredients = meal['Ingrediets']
    ignored = set()  # Store ignored items here if needed

    diet_frame = find_best_match(ingredients, ignored)

    meals_list.append({
        'calories': meal['meal_calories_sum'],
        'meal_type': meal['meal_type'],
        'title': meal['title'],
        'steps': meal['steps'],
        'data_frame': diet_frame
    })

updated_meal_data_frames = {}
for diet_frame in meals_list:
    updated_meal_data_frames[diet_frame['meal_type']] = \
    {'title': diet_frame['title'],
    'steps': diet_frame['steps'],
    'Ingrediets': ratios_adjustment_data_frame(diet_frame_gpt = diet_frame['data_frame'], calories_gpt = diet_frame['calories'])}

AttributeError: ignored

In [1]:
diet_frame_gpt = diet_frame['data_frame']
calories_gpt = diet_frame['calories']

NameError: name 'diet_frame' is not defined

In [ ]:
diet_frame_gpt

,name,calories,mass,food_group,Name,Calories,Food Group,Fat (g),Protein (g),Carbohydrate (g),200 Calorie Weight (g),pr_cl,ft_cl,cb_cl,prot_rt,fat_rt,carb_rt
0,Eggs,70,2 large,Protein,Eggs (Raw),143.0,Dairy and Egg Products,9.51,12.56,0.72,139.860,50.24,85.59,2.88,0.362195,0.617043,0.020763
1,Cheese,110,1/4 cup,Fat,Cheese Sauce,160.0,Soups and Sauces,13.02,3.24,7.66,125.000,12.96,117.18,30.64,0.080607,0.728822,0.190571
2,Spinach,10,1 cup,Vegetables,Spinach,23.0,Vegetables,0.39,2.86,3.63,869.565,11.44,3.51,14.52,0.388191,0.119104,0.492704
3,Tomatoes,25,1 medium,Fruit,Tomatoes,18.0,Vegetables,0.20,0.88,3.89,1111.111,3.52,1.80,15.56,0.168582,0.086207,0.745211


In [ ]:
diet_frame_gpt.loc[:, 'pr_cl'] = diet_frame_gpt['Protein (g)'] * 4
diet_frame_gpt.loc[:, 'ft_cl'] = diet_frame_gpt['Fat (g)'] * 9
diet_frame_gpt.loc[:, 'cb_cl'] = diet_frame_gpt['Carbohydrate (g)'] * 4
diet_frame_gpt['prot_rt'] = diet_frame_gpt['pr_cl']/(diet_frame_gpt['ft_cl']+diet_frame_gpt['pr_cl']+diet_frame_gpt['cb_cl'])
diet_frame_gpt['fat_rt'] = diet_frame_gpt['ft_cl']/(diet_frame_gpt['ft_cl']+diet_frame_gpt['pr_cl']+diet_frame_gpt['cb_cl'])
diet_frame_gpt['carb_rt'] = diet_frame_gpt['cb_cl']/(diet_frame_gpt['ft_cl']+diet_frame_gpt['pr_cl']+diet_frame_gpt['cb_cl'])
ingreds = list(diet_frame_gpt['name'])
# ingreds_adjustable = find_contributer_ingreds(diet_frame_gpt)
ingreds_adjustable = ["Eggs", "Cheese", "Tomatoes"]
diet_frame = pd.DataFrame(diet_frame_gpt.loc[diet_frame_gpt['name'].isin(ingreds_adjustable)].reset_index(drop=True))
excluded_diet_frame = diet_frame_gpt.loc[~diet_frame_gpt['name'].isin(ingreds_adjustable)].reset_index(drop=True)
ingredients_data = [Ingredient_class(diet_frame['name'][i],
                                diet_frame['prot_rt'][i],
                                diet_frame['fat_rt'][i],
                                diet_frame['carb_rt'][i])
                    for i in range(len(diet_frame))]
# calories = sum(diet_frame['calories'])
calories = calories_gpt
required_protein = (macros_ratio[activity_factor]['Protein']/100)*(calories)
required_fat = (macros_ratio[activity_factor]['Fat']/100)*(calories)
required_carb = (macros_ratio[activity_factor]['Carbohydrates']/100)*(calories)
remaining_required_protein = required_protein - sum(excluded_diet_frame['prot_rt']*excluded_diet_frame['calories'])
remaining_required_fat = required_fat - sum(excluded_diet_frame['fat_rt']*excluded_diet_frame['calories'])
remaining_required_carb = required_carb - sum(excluded_diet_frame['carb_rt']*excluded_diet_frame['calories'])
min_max_bounds = {diet_frame['name'][i]: diet_frame['calories'][i]*0.5
                    for i in range(len(diet_frame))}
tolerance = 5
adjusted_ingredients = adjust_ingredients(ingredients_data,
                                          remaining_required_protein, remaining_required_fat, remaining_required_carb,
                                          min_max_bounds)
for name, amount in adjusted_ingredients.items():
    diet_frame_gpt.loc[diet_frame_gpt['name']==name, 'calories'] = amount
    print(f"{name}: {amount:.2f} calories")
# Verify results
if verify_results(adjusted_ingredients, ingredients_data, remaining_required_protein, remaining_required_fat, remaining_required_carb):
    print("\nAchieved macros are within the desired range!")
else:
    print("\nAchieved macros are not within the desired range.")

diet_frame_gpt['1cal_gm'] = 100/diet_frame_gpt['Calories']
temp_cal = sum(diet_frame_gpt['calories'])
diet_frame_gpt['cal_ratio'] = diet_frame_gpt['calories']/temp_cal
diet_frame_gpt['required_calories'] = ((calories)*diet_frame_gpt['cal_ratio'])
diet_frame_gpt['per_meal_gms'] = diet_frame_gpt['required_calories'] * diet_frame_gpt['1cal_gm']
diet_frame_gpt['taste'] = ''
diet_frame_gpt.loc[diet_frame_gpt['required_calories']==0.0, 'taste'] = 'As per taste'
df = pd.DataFrame(diet_frame_gpt[['Name', 'per_meal_gms']])
result = []
for index, row in df.iterrows():
    result.append({k: v for k, v in row.items()})

Eggs: 159.02 calories
Cheese: 0.86 calories
Tomatoes: 290.67 calories

Achieved macros are not within the desired range.


In [ ]:
remaining_required_protein

141.9035268023719

In [ ]:
remaining_required_fat

121.65378584884687

In [ ]:
remaining_required_carb

201.44268734878122

In [ ]:
diet_frame_gpt

,name,calories,mass,food_group,Name,Calories,Food Group,Fat (g),Protein (g),Carbohydrate (g),200 Calorie Weight (g),pr_cl,ft_cl,cb_cl,prot_rt,fat_rt,carb_rt,1cal_gm,cal_ratio,required_calories,per_meal_gms,taste
0,Eggs,185.85317,2 large,Protein,Eggs (Raw),143.0,Dairy and Egg Products,9.51,12.56,0.72,139.860,50.24,85.59,2.88,0.362195,0.617043,0.020763,0.699301,0.816118,408.059244,285.356115,
1,Cheese,6.87500,1/4 cup,Fat,Cheese Sauce,160.0,Soups and Sauces,13.02,3.24,7.66,125.000,12.96,117.18,30.64,0.080607,0.728822,0.190571,0.625000,0.030190,15.094751,9.434219,
2,Spinach,10.00000,1 cup,Vegetables,Spinach,23.0,Vegetables,0.39,2.86,3.63,869.565,11.44,3.51,14.52,0.388191,0.119104,0.492704,4.347826,0.043912,21.956001,95.460875,
3,Tomatoes,25.00000,1 medium,Fruit,Tomatoes,18.0,Vegetables,0.20,0.88,3.89,1111.111,3.52,1.80,15.56,0.168582,0.086207,0.745211,5.555556,0.109780,54.890003,304.944463,


In [ ]:
ingreds_adjustable

['Cheese', 'Eggs']

In [ ]:
remaining_required_fat

119.69

In [ ]:
remaining_required_carb

194.92000000000002

In [ ]:
a= find_contributer_ingreds(diet_frame_gpt)

Eggs


In [ ]:
diet_frame_gpt

,name,calories,mass,food_group,Name,Calories,Food Group,Fat (g),Protein (g),Carbohydrate (g),200 Calorie Weight (g),pr_cl,ft_cl,cb_cl,prot_rt,fat_rt,carb_rt,1cal_gm,cal_ratio,required_calories,per_meal_gms,taste
0,Eggs,35,2 large,Protein,Eggs (Raw),143.0,Dairy and Egg Products,9.51,12.56,0.72,139.860,50.24,85.59,2.88,0.362195,0.617043,0.020763,0.699301,0.28,50.4,35.244755,
1,Cheese,55,1/4 cup,Fat,Cheese Sauce,160.0,Soups and Sauces,13.02,3.24,7.66,125.000,12.96,117.18,30.64,0.080607,0.728822,0.190571,0.625000,0.44,79.2,49.500000,
2,Spinach,10,1 cup,Vegetables,Spinach,23.0,Vegetables,0.39,2.86,3.63,869.565,11.44,3.51,14.52,0.388191,0.119104,0.492704,4.347826,0.08,14.4,62.608696,
3,Tomatoes,25,1 medium,Fruit,Tomatoes,18.0,Vegetables,0.20,0.88,3.89,1111.111,3.52,1.80,15.56,0.168582,0.086207,0.745211,5.555556,0.20,36.0,200.000000,


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def find_sweet_spot(df, col1, col2, weight_col1=0.5, weight_col2=0.5):
    """
    Find the row that maximizes both criteria simultaneously for two columns with different scales
    by considering a weighted combination of the criteria.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - col1 (str): Name of the first column.
    - col2 (str): Name of the second column.
    - weight_col1 (float, optional): Weight for the first column (default is 0.5).
    - weight_col2 (float, optional): Weight for the second column (default is 0.5).

    Returns:
    - pd.Series: The row that maximizes both criteria.
    """
    # Initialize Min-Max scaler
    scaler = MinMaxScaler()

    # Scale the columns
    df_scaled = df.copy()
    df_scaled[[col1, col2]] = scaler.fit_transform(df[[col1, col2]])

    # Calculate a combined score based on the weighted sum of the two criteria
    df_scaled['Combined_Score'] = (df_scaled[col1] * weight_col1) + (df_scaled[col2] * weight_col2)

    # Find the row with the highest combined score
    sweet_spot_row = df_scaled[df_scaled['Combined_Score'] == df_scaled['Combined_Score'].max()]

    return sweet_spot_row



# Find the sweet spot row that maximizes both criteria with weighted combination and scaling
sweet_spot = find_sweet_spot(diet_frame_gpt, 'Fat (g)', 'fat_rt')

# Print the sweet spot row
print(sweet_spot)


     name  calories     mass food_group          Name  Calories  \
1  Cheese  0.859375  1/4 cup        Fat  Cheese Sauce     160.0   

         Food Group  Fat (g)  Protein (g)  Carbohydrate (g)  \
1  Soups and Sauces      1.0         3.24              7.66   

   200 Calorie Weight (g)  pr_cl   ft_cl  cb_cl   prot_rt  fat_rt   carb_rt  \
1                   125.0  12.96  117.18  30.64  0.080607     1.0  0.190571   

   1cal_gm  cal_ratio  required_calories  per_meal_gms taste  Combined_Score  
1    0.625   0.001866           0.932987      0.583117                   1.0  


In [ ]:
def find_contributer_ingreds(ingreds):
    print('finding suitable ingreds')
    prompt_for_ingreds = f"I have ingredients {', '.join(set(ingreds))} please give which one ingredient would you choose to adjust carb quantity in this meal without affecting meal taste and texture that much"
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
       {"role": "user", "content": prompt_for_ingreds}
            ]
        )
    ingreds_adjustable = response.choices[0]["message"]["content"]
    return ingreds_adjustable

In [ ]:
find_contributer_ingreds(['Yogurt','Protein Powder','Oats', 'Berries', 'honey'])

finding suitable ingreds


'To adjust the carb quantity in this meal without significantly affecting the taste and texture, I would choose to adjust the oats. Here are a few ways you can modify the oats to control the carb content:\n\n1. Reduce the amount of oats: Simply decrease the quantity of oats you use in the meal to reduce the carb content. This will have minimal impact on the taste and texture, as long as the ratio of other ingredients remains balanced.\n\n2. Choose low-carb oats: Opt for low-carb oats, such as steel-cut oats or oat bran, instead of traditional rolled oats. These varieties tend to have a lower carb content while maintaining a similar texture.\n\n3. Use a 50/50 mix: Replace half of the oats with a lower-carb alternative, like crushed nuts or ground flaxseed. This combination will help reduce the overall carb content while maintaining the desired texture and flavor.\n\nRemember to adjust the quantities of other ingredients accordingly to keep the desired taste and balance within the meal.'

In [ ]:


def find_contributer_ingreds(df):
    # Find the ingredient with the highest protein content
    highest_protein_ingredient = df.loc[df['Protein (g)'].idxmax()]['name']
    print(highest_protein_ingredient)
    # Find the ingredient with the highest fat content
    highest_fat_ingredient = df.loc[df['Fat (g)'].idxmax()]['name']

    # Find the ingredient with the highest carbohydrate content
    highest_carb_ingredient = df.loc[df['Carbohydrate (g)'].idxmax()]['name']
    return list(np.unique([highest_protein_ingredient, highest_fat_ingredient, highest_carb_ingredient]))

In [ ]:
diet_frame_gpt.loc[diet_frame_gpt['prot_rt'].idxmax(),'name']

'Spinach'

In [ ]:
def calculate_ingredient_quantities(required_calories, required_proteins, required_fats, required_carbs):
    # Define the macronutrient composition of each ingredient per unit
    protein_source_composition = {'Protein': 20, 'Fat': 5, 'Carbs': 10, 'Calories': 150}
    carb_source_composition = {'Protein': 2, 'Fat': 1, 'Carbs': 30, 'Calories': 120}
    fat_source_composition = {'Protein': 0, 'Fat': 10, 'Carbs': 0, 'Calories': 90}

    # Initialize quantities of each ingredient to 0
    protein_source_quantity = 0
    carb_source_quantity = 0
    fat_source_quantity = 0

    # Initialize total calories, total proteins, total fats, and total carbs
    total_calories = 0
    total_proteins = 0
    total_fats = 0
    total_carbs = 0

    # Define a small increment for adjusting ingredient quantities
    increment = 1

    # Iterate until goals are met or a reasonable number of iterations is reached
    max_iterations = 1000
    for _ in range(max_iterations):
        # Calculate the current totals
        current_calories = (protein_source_quantity * protein_source_composition['Calories'] +
                            carb_source_quantity * carb_source_composition['Calories'] +
                            fat_source_quantity * fat_source_composition['Calories'])
        current_proteins = (protein_source_quantity * protein_source_composition['Protein'] +
                            carb_source_quantity * carb_source_composition['Protein'] +
                            fat_source_quantity * fat_source_composition['Protein'])
        current_fats = (protein_source_quantity * protein_source_composition['Fat'] +
                        carb_source_quantity * carb_source_composition['Fat'] +
                        fat_source_quantity * fat_source_composition['Fat'])
        current_carbs = (protein_source_quantity * protein_source_composition['Carbs'] +
                         carb_source_quantity * carb_source_composition['Carbs'] +
                         fat_source_quantity * fat_source_composition['Carbs'])

        # Check priorities and adjust quantities accordingly
        if total_calories < required_calories:
            if current_proteins < required_proteins:
                protein_source_quantity += increment
            elif current_fats < required_fats:
                fat_source_quantity += increment
            else:
                carb_source_quantity += increment
        else:
            # Goals are met
            break

        # Update total calories, proteins, fats, and carbs
        total_calories = current_calories
        total_proteins = current_proteins
        total_fats = current_fats
        total_carbs = current_carbs

    return {
        'Protein Source': protein_source_quantity,
        'Carb Source': carb_source_quantity,
        'Fat Source': fat_source_quantity,
        'Total Calories': total_calories,
        'Total Proteins': total_proteins,
        'Total Fats': total_fats,
        'Total Carbs': total_carbs
    }

# Example usage:
# Define the required nutritional goals
required_calories = 2000
required_proteins = 150
required_fats = 60
required_carbs = 250

# Calculate ingredient quantities to meet the goals
ingredient_quantities = calculate_ingredient_quantities(required_calories, required_proteins, required_fats, required_carbs)
print(ingredient_quantities)


{'Protein Source': 8, 'Carb Source': 7, 'Fat Source': 2, 'Total Calories': 2100, 'Total Proteins': 172, 'Total Fats': 66, 'Total Carbs': 260}


In [ ]:
def calculate_ingredient_quantities_bulking(required_calories, required_proteins, required_fats, required_carbs):
    # Define the macronutrient composition of each ingredient per unit
    protein_source_composition = {'Protein': 20, 'Fat': 5, 'Carbs': 10, 'Calories': 150}
    carb_source_composition = {'Protein': 2, 'Fat': 1, 'Carbs': 30, 'Calories': 120}
    fat_source_composition = {'Protein': 0, 'Fat': 10, 'Carbs': 0, 'Calories': 90}

    # Initialize quantities of each ingredient to 0
    protein_source_quantity = 0
    carb_source_quantity = 0
    fat_source_quantity = 0

    # Initialize total calories, total proteins, total fats, and total carbs
    total_calories = 0
    total_proteins = 0
    total_fats = 0
    total_carbs = 0

    # Define a small increment for adjusting ingredient quantities
    increment = 1

    # Iterate until goals are met or a reasonable number of iterations is reached
    max_iterations = 1000
    for _ in range(max_iterations):
        # Calculate the current totals
        current_calories = (protein_source_quantity * protein_source_composition['Calories'] +
                            carb_source_quantity * carb_source_composition['Calories'] +
                            fat_source_quantity * fat_source_composition['Calories'])
        current_proteins = (protein_source_quantity * protein_source_composition['Protein'] +
                            carb_source_quantity * carb_source_composition['Protein'] +
                            fat_source_quantity * fat_source_composition['Protein'])
        current_fats = (protein_source_quantity * protein_source_composition['Fat'] +
                        carb_source_quantity * carb_source_composition['Fat'] +
                        fat_source_quantity * fat_source_composition['Fat'])
        current_carbs = (protein_source_quantity * protein_source_composition['Carbs'] +
                         carb_source_quantity * carb_source_composition['Carbs'] +
                         fat_source_quantity * fat_source_composition['Carbs'])

        # Check priorities and adjust quantities accordingly
        if total_proteins < required_proteins:
            protein_source_quantity += increment
        elif total_calories < required_calories:
            if current_fats < required_fats:
                fat_source_quantity += increment
            else:
                carb_source_quantity += increment
        else:
            # Goals are met
            break

        # Update total calories, proteins, fats, and carbs
        total_calories = current_calories
        total_proteins = current_proteins
        total_fats = current_fats
        total_carbs = current_carbs

    return {
        'Protein Source': protein_source_quantity,
        'Carb Source': carb_source_quantity,
        'Fat Source': fat_source_quantity,
        'Total Calories': total_calories,
        'Total Proteins': total_proteins,
        'Total Fats': total_fats,
        'Total Carbs': total_carbs
    }

# Example usage:
# Define the required nutritional goals for bulking
required_calories_bulking = 3000
required_proteins_bulking = 180
required_fats_bulking = 80
required_carbs_bulking = 400

# Calculate ingredient quantities for bulking to meet the goals
ingredient_quantities_bulking = calculate_ingredient_quantities_bulking(
    required_calories_bulking, required_proteins_bulking, required_fats_bulking, required_carbs_bulking)
print(ingredient_quantities_bulking)


{'Protein Source': 10, 'Carb Source': 12, 'Fat Source': 3, 'Total Calories': 3090, 'Total Proteins': 222, 'Total Fats': 91, 'Total Carbs': 430}


In [1]:
import numpy as np
list(np.unique({1:'er', 2: 'mnm', 1:'dd'}))

[{1: 'dd', 2: 'mnm'}]

In [2]:
{1:'er', 2: 'mnm', 1:'dd'}

{1: 'dd', 2: 'mnm'}